# Extracting parameters

In [1]:
import numpy as np
import time
import qutip as qtp
import matplotlib.pyplot as plt
import scipy.integrate
from scipy.optimize import minimize
import pysqkit
from pysqkit import QubitSystem
from pysqkit.util.metrics import average_process_fidelity, \
    average_gate_fidelity
from pysqkit.drives.pulse_shapes import gaussian_top
from pysqkit.util.phys import temperature_to_thermalenergy
from pysqkit.util.quantum import generalized_rabi_frequency
import pysqkit.util.transformations as trf
from pysqkit.util.hsbasis import weyl_by_index
from typing import List, Dict, Callable
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
import cmath
import json

from IPython.display import display, Latex

In [2]:
with open('flx_transm_params.txt') as param_file:
    parameters_set = json.load(param_file)

In [3]:
temperature = 0.020 # K
thermal_energy = temperature_to_thermalenergy(temperature) # kb T/h in GHz
d_comp = 4

p_set = "0"


#Transmon
levels_t = 3
transm = pysqkit.qubits.SimpleTransmon(
    label='T', 
    max_freq=parameters_set[p_set]["max_freq_t"] + 1.3, 
    anharm=parameters_set[p_set]["anharm_t"],
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_t"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,    
    dim_hilbert=levels_t,
    dephasing_times=parameters_set[p_set]["dephasing_times_t"]
)

#Fluxonium
levels_f = 5

flx = pysqkit.qubits.Fluxonium(
    label='F', 
    charge_energy=parameters_set[p_set]["charge_energy_f"], 
    induct_energy=parameters_set[p_set]["induct_energy_f"], 
    joseph_energy=parameters_set[p_set]["joseph_energy_f"], #8.0, 
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_f"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,
    dephasing_times= parameters_set[p_set]["dephasing_times_f"] #ns/2*np.pi 
)
flx.diagonalize_basis(levels_f)

# We also add a drive on the fluxonium
flx.add_drive(
    pysqkit.drives.microwave_drive,
    label='cz_drive_f',
    pulse=pysqkit.drives.pulses.cos_modulation,
    pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
)

d_leak = levels_t*levels_f - d_comp

jc = parameters_set[p_set]["jc"]
coupled_sys = transm.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
bare_system = transm.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=0.0)

freq_flx = flx.state('1')[0] - flx.state('0')[0]
freq_03_flx = flx.state('3')[0] - flx.state('0')[0]
freq_34_flx = flx.state('4')[0] - flx.state('3')[0]

In [4]:
parameters_set[p_set]

{'max_freq_t': 4.37,
 'anharm_t': -0.3,
 'diel_loss_tan_t': 3e-07,
 'dephasing_times_t': {'1': 314159.26535897935, '2': 188495.55921538756},
 'charge_energy_f': 1.0,
 'induct_energy_f': 0.5,
 'joseph_energy_f': 8.0,
 'diel_loss_tan_f': 3.5e-06,
 'dephasing_times_f': {'1': 1256637.0614359174,
  '2': 125663.70614359173,
  '3': 125663.70614359173,
  '4': 31415.926535897932},
 'jc': 0.03}

In [5]:
def zz(system: QubitSystem) -> float:
    xi_zz = system.state('00')[0] + system.state('11')[0] \
        - system.state('01')[0] - system.state('10')[0]
    return xi_zz

def delta(system: QubitSystem) -> float:
    delta_gate = (system.state('13')[0] - system.state('10')[0]) - \
        (system.state('03')[0] - system.state('00')[0])
    return delta_gate 

In [6]:
display(Latex(r'$\omega_{{01, flx}}/2 \pi = {:.3f} \, \mathrm{{GHz}}$'.format(freq_flx)))
display(Latex(r'$\omega_{{01, transm}}/2 \pi = {:.3f} \, \mathrm{{GHz}}$'.format(transm.max_freq)))
display(Latex(r'$\omega_{{34, flx}}/2 \pi = {:.3f} \, \mathrm{{GHz}}$'.format(freq_34_flx)))
display(Latex(r'$\omega_{{03, flx}}/2 \pi = {:.3f} \, \mathrm{{GHz}}$'.format(freq_03_flx)))
display(Latex(r'$\xi_{{ZZ}}/h = {:.3f} \, \mathrm{{MHz}}$'.format(1e3*zz(coupled_sys))))
display(Latex(r'$\Delta/h = {:.3f} \, \mathrm{{MHz}}$'.format(1e3*delta(coupled_sys))))
display(Latex(r'$T_{{1,transm}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*transm.loss_rates(0, 1)[0]))))
display(Latex(r'$T_{{1,flx}}^{{1 \mapsto 0}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*flx.loss_rates(0, 1)[0]))))
display(Latex(r'$T_{{1,flx}}^{{0 \mapsto 1}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*flx.loss_rates(0, 1)[1]))))
display(Latex(r'$T_{{1,flx}}^{{3 \mapsto 0}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*flx.loss_rates(0, 3)[0]))))
display(Latex(r'$T_{{1,flx}}^{{0 \mapsto 3}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*flx.loss_rates(0, 3)[1]))))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>